# Interactive plotting with Bokeh

[reference](https://docs.bokeh.org/en/latest/docs/user_guide/interaction.html)

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook() # alternatively one could use output_file()

Loading BokehJS ...

## ColumnDataSource (CDS)
([reference](https://docs.bokeh.org/en/latest/docs/user_guide/data.html))

- Useful for most interactive plotting
- Can also take ```pandas``` ```DataFrame```s (not compulsory)
- Required for [data streaming](https://docs.bokeh.org/en/latest/docs/user_guide/data.html#ajaxdatasource) (not covered in this talk)

## Layouts
([reference](hhttps://docs.bokeh.org/en/latest/docs/user_guide/layout.html))

- Describe how the plots are displayed
- Following possibilities:
    - [column](https://docs.bokeh.org/en/latest/docs/user_guide/layout.html#column-layout)
    - [row](https://docs.bokeh.org/en/latest/docs/user_guide/layout.html#row-layout)
    - [grid](https://docs.bokeh.org/en/latest/docs/user_guide/layout.html#grid-layout-for-plots)
    - [general](https://docs.bokeh.org/en/latest/docs/user_guide/layout.html#general-grid-layout)

#### Line plot with a CDS

In [2]:
import numpy as np
from bokeh.models import BooleanFilter, CDSView, ColumnDataSource as CDS

#create CDS
npoints = 100
dline = dict(x=np.arange(npoints), 
             y=np.random.normal(0, 0.5, size=npoints),
            )
source = CDS(data=dline)

#create CDS view (filtered CDS)
positives = [True if k>0 else False for k in source.data['y']]
view = CDSView(source=source, filters=[BooleanFilter(positives)])

#figure options
TOOLS = "pan,hover,box_select,lasso_select,help"
fig_options = dict(plot_width=400, plot_height=400, tools=TOOLS)
marker_options = dict(color='red', 
                      fill_color='orange', fill_alpha=0.8)

#create figure with specifically chosen tools
pleft = figure(**fig_options)
pleft.scatter('x', 'y', source=source, **marker_options)

#create another figure using the view
pright = figure(y_range=pleft.y_range, **fig_options)
pright.circle('x', 'y', source=source, view=view, **marker_options)

GlyphRenderer(id='1073', ...)

In [3]:
#introducing layouts
from bokeh.layouts import row
lay = row(pleft, pright)
show(lay)

There are three ways to build interactive plots with ```bokeh```:
- the one you just saw
    - <span style="color:green">very straightforward</span> / <span style="color:red">very limited</span>
- use widgets together with the ```bokeh``` server
    - <span style="color:green">much more flexible, Python only</span> / <span style="color:red">requires a running server, some limitations</span>
- use widgets together with Javascript callbacks
    - <span style="color:green">"infinitely" flexible</span> / <span style="color:red">requires some Javascript</span> (in general only a bit)

I personally feel the last one to be the best for almost all scenarios:
- I could easily grasp the required Javscript although I did not know the language
- You can do virtually anything with it

## Using ```CustomJS``` callbacks

[reference](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html)

In [4]:
#many widgets available: https://docs.bokeh.org/en/latest/docs/user_guide/interaction/widgets.html
from bokeh.models import CustomJS, Slider


In [5]:
slider = Slider(start=1, end=40, value=5, step=1)
show(slider)

Let us add some data to the CDS:

In [6]:
def print_cds_columns(s):
    for i,col in enumerate(s.data):
        print('CDS column #{}: {}'.format(i,col))

source.data['size'] = np.ones(npoints)*10
print_cds_columns(source)

CDS column #0: x
CDS column #1: y
CDS column #2: size


In [7]:
pint = figure(**fig_options)
pint.circle('x', 'y', size='size', source=source, **marker_options)

GlyphRenderer(id='1265', ...)

In [8]:
code = """
    var val = slider.value;
    var data = s.data;
    for (var i = 0; i<data['size'].length; i++) {{
        data['size'][i] = val;
    }}
    s.change.emit();
"""

callback = CustomJS(args=dict(s=source, slider=slider), 
                    code=code)

slider.title = 'Points size'
slider.js_on_change('value', callback)

In [9]:
from bokeh.layouts import layout
lay = layout([[slider],[pint]])
show(lay)

In [ ]:
code = """
    var from_exp = rs_exp.value[0];
    var from_gau = rs_gau.value[0];
    var from_exp2 = rs_exp2.value[0];
    var from_gau2 = rs_gau2.value[0];
    
    var to_exp = rs_exp.value[1];
    var to_gau = rs_gau.value[1];
    var to_exp2 = rs_exp2.value[1];
    var to_gau2 = rs_gau2.value[1];
    
    var d_exp  = s.data['exp'];
    var d_gau  = s.data['gau'];
    var d_exp2 = s.data['exp2'];
    var d_gau2 = s.data['gau2'];
    
    var dup_exp  = sup.data['exp'];
    var dup_gau  = sup.data['gau'];
    var dup_exp2 = sup.data['exp2'];
    var dup_gau2 = sup.data['gau2'];
    dup_exp  = [];
    dup_gau  = [];
    dup_exp2 = [];
    dup_gau2 = [];
    
    for (var i=0; i<d_exp.length; i++) {
        if (d_exp[i] > from_exp && d_exp[i] < to_exp) {
            dup_exp.push(d_exp[i])
        }
        if (d_gau[i] > from_gau && d_gau[i] < to_gau) {
            dup_gau.push(d_gau[i])
        }
        if (d_exp2[i] > from_exp2 && d_exp2[i] < to_exp2) {
            dup_exp2.push(d_exp2[i])
        }
        if (d_gau2[i] > from_gau2 && d_gau2[i] < to_gau2) {
            d_gau2.push(d_gau2[i])
        }        
    }   
        
    sup.change.emit();
    s.change.emit();
"""

callback = CustomJS(args=dict(s=source, sup=source_update,
                              rs_exp=rs_exp, rs_gau=rs_gau,
                              rs_exp2=rs_exp2, rs_gau2=rs_gau2), 
                    code=code)

rs_exp.js_on_change('value', callback)
rs_gau.js_on_change('value', callback)
rs_exp2.js_on_change('value', callback)
rs_gau2.js_on_change('value', callback)

### Examples by the community:

- [Bioinformatics](https://pirovc.github.io/grimer-reports/others/placenta_wgs.html)
- [COVID interactive paper](https://jacob-barhak.github.io/COVID19_Ensemble_2021.html)
- [Fitness](https://hnagib.com/)
- [Mortality rates](https://cjdixon.s3-ap-southeast-2.amazonaws.com/bokeh/mortality_rates.html)
- [Simple simulation](https://polyas-urn.herokuapp.com/app)
 